# Exploring Simon's Periodicity Algorithm: Quantum Component

**Simon's periodicity algorithm** is a well-known oracle-based quantum computing algorithm about finding patterns in functions. It is famous for solving [Simon's problem](https://en.wikipedia.org/wiki/Simon%27s_problem) exponentially faster than any known classical algorithm. Simon's algorithm contains both quantum and classical procedures. While the algorithm itself has little practical value, it is a useful learning tool for illustrating important concepts like quantum oracles. As such, Simon's algorithm is part of many introductory courses on quantum computing.

This tutorial will:
* introduce you to the problem solved by Simon's algorithm,
* walk you through the overview of the algorithm,
* give you practice implementing relevant quantum oracles using the Q# programming language, and
* give you practice implementing the quantum circuit for Simon's algorithm.

In the last section of the tutorial you will continue exploration of Simon's algorithm in a [companion Python notebook](./ExploringSimonsAlgorithmClassicalPart.ipynb) that will introduce you to the classical component of the algorithm.

Let's go!

To begin, first prepare this notebook for execution (if you skip the first step, you'll get the "Syntax does not match any known patterns" error when you try to execute Q# code in the next cells; if you skip the second step, you'll get the "Invalid test name" error).

Click the cell with code below this block of text and press `Ctrl+Enter` (`shift+return` on Mac).

In [9]:
%package Microsoft.Quantum.Katas::0.11.2003.3107

Microsoft.Quantum.Standard::0.10.1912.0501, Microsoft.Quantum.Katas::0.10.1912.0501

> **Note:** The package versions in the output of the cell above should always match. If you are running the Notebooks locally and the versions do not match, please install the IQ# version that matches the version of the Microsoft.Quantum.Katas package.

In [11]:
%workspace reload

Quantum.Kata.ExploringSimonsAlgorithm.ApplyOracleWithOutputArrA, Quantum.Kata.ExploringSimonsAlgorithm.AssertTwoOraclesWithOutputArrAreEqual, Quantum.Kata.ExploringSimonsAlgorithm.Bitwise_LeftShift_Oracle, Quantum.Kata.ExploringSimonsAlgorithm.Bitwise_LeftShift_Oracle_Reference, Quantum.Kata.ExploringSimonsAlgorithm.E1_QuantumOracle_Test, Quantum.Kata.ExploringSimonsAlgorithm.Simon_Algorithm, Quantum.Kata.ExploringSimonsAlgorithm.Simons_Algorithm_Reference

# Part I. Problem Statement

## Simon's Problem

**You are given a classical function $f: \{0, 1\}^n\ \to\ \{0, 1\}^n$ as a black box. The task is to determine $\mathbf{s}$, the period of $f$.**

Further, you are assured that there exists a secret set of $n$ boolean variables $\mathbf{s} = (s_0, s_1, ..., s_{n-1})$ such that for all $\mathbf{x}$, $\mathbf{y} \in\ \{0, 1\}^n$,

$$f(\mathbf{x}) = f(\mathbf{y}) \text{ if and only if } \mathbf{x} = \mathbf{y} \oplus \mathbf{s},$$
where $\oplus$ is the bitwise exclusive-or operation.

In other words, the values of $f$ repeat themselves in some pattern and the pattern is determined by $\mathbf{s}$.

## Example
Let $n=3$. The following function satisfies the required and just mentioned property:

| $x$ | $f(x)$ |
| ------- | ----- |
| 000 | 000 |
| 001 | 010 |
| 010 | 100 |
| 011 | 110 |
| 100 | 000 |
| 101 | 010 |
| 110 | 100 |
| 111 | 110 |

It can be seen that each output $f(x)$ occurs twice. As such, $f$ is two-to-one. We can obtain $\mathbf{s}$ by performing the bitwise XOR operation between both inputs that are mapped to the same output. That is,

$000 \oplus 100 = 100 = \mathbf{s}$,

$001 \oplus 101 = 100 = \mathbf{s}$, and so on.

## Classical solution

If we solve this problem classically, how many evaluations of $f$ will we need?

We need to find two different inputs $x$ and $y$ such that $f(\mathbf{x}) = f(\mathbf{y})$. If we do not know anything about the internal structure of $f$, we would have to evaluate the function on different inputs. After each evaluation, we would check to see if that output has already been found. If we find two such inputs, we know that

$\mathbf{x} = \mathbf{y} \oplus \mathbf{s}$.

We can obtain $\mathbf{s}$ by $\oplus$-ing both sides with $\mathbf{y}$:

$\mathbf{x} \oplus \mathbf{y} = \mathbf{y} \oplus \mathbf{s} \oplus \mathbf{y} = \mathbf{s}$.

To find a pair for which $f$ takes the same output, we would likely need to guess $\Omega(\sqrt{2^n})$ different inputs.

# Part II. Oracles

Like the Deutsch-Jozsa algorithm you learned previously in the [Exploring Deutsch-Jozsa Algorithm tutorial](https://github.com/microsoft/QuantumKatas/tree/24002be02c034354356c92a31ba4a99ac897cbe6/tutorials/ExploringDeutschJozsaAlgorithm), Simon's algorithm utilizes quantum oracles. Unlike D-J, however, these are *not* phase oracles and they act on multiple qubits. 

Let's see an example of how a multi-qubit quantum oracle is implemented in Q#.

## Example: Multi-qubit quantum oracle

>**Note:** This code snippet is an example. It does not need to be modified and is not covered by tests.

**Inputs:**

1. $2$ qubits in an arbitrary state $|x\rangle$ (input/query register)
2. $2$ qubits in the state $|00\rangle$ (target register)

**Goal:**

Set the target qubits by applying a function such that they represent the following transformation on the input qubits:
* If the query register is in the state $|00\rangle$, the target register should become $|11\rangle$
* If the query register is in the state $|01\rangle$, the target register should become $|10\rangle$
* If the query register is in the state $|10\rangle$, the target register should become $|01\rangle$
* If the query register is in the state $|11\rangle$, the target register should become $|00\rangle$

In [3]:
operation MultiQubit_Oracle (x : Qubit[], y: Qubit[]) : Unit is Adj{
    // Can also hardcode N equal to 2 for this example
    let N = Length(x);
    
    // This is a Not gate controlled on 0
    for (i in 0 .. N - 1) {
        (ControlledOnBitString([false], X))([x[i]], y[i]);
    }
}

MultiQubit_Oracle

> Note that the previous oracle implementation didn't change the state of the qubits in the query register; it only changed those in the target register.

Now that you have seen an example of a multi-qubit oracle implemenatation, try implementing the quantum oracle for the example problem above where $\mathbf{s} = 100$. In other words, how can you go from each state represented in the $x$ column to its corresponding state in the $f(x)$ column?

## Exercise 1: Oracle for example where $\mathbf{s} = 100$, $n = 3$

**Inputs:**

1. $3$ qubits in an arbitrary state $|x\rangle$ (input/query register)
2. $3$ qubits in an arbitrary state $|y\rangle$ (target register)

**Goal:**

Transform state $|x, y\rangle$ into $|x, y \oplus f(x)\rangle$, where $f$ is the bitwise left shift function; i.e.
$|y \oplus f(x)\rangle = |y_0 \oplus x_1, y_1 \oplus x_2, ..., y_{n-1} \oplus x_{n}\rangle$.

**Hint:** As in the multi-qubit oracle example, you should only change the state of the qubits in the target register.

In [ ]:
%kata E1_QuantumOracle_Test

operation Bitwise_LeftShift_Oracle (x : Qubit[], y: Qubit[]) : Unit is Adj{
    //...
}

# Part III. Simon's Algorithm

### Input

We are given two multi-dimensional qubit registers as input in the familiar form $|0^n\rangle \otimes |0^n\rangle = |0^n\rangle|0^n\rangle$, where one is a data register and the other a target register.

### Algorithm summary

Simon's algorithm consists of a quantum procedure followed by classical post-processing. The post-processing needed for step 5 is discussed in the accompanying tutorial. The steps of the algorithm can be summarized as follows:

1. Apply the Hadamard transform to the first $n$ qubits.
2. Call the oracle "black box" $U_{f}$ to compute $f$ on the transformed input.
3. Apply a second Hadamard transform to the states of the first $n$ qubits.
4. Measure all qubits of the input register.
5. Repeat the previous four steps $n-1$ times.
5. Solve system of linear equations in step 4 to obtain $\mathbf{s}$.

> **Note:** If you need an introduction to or refresher on performing measurements, check out the [Measurements kata](https://github.com/microsoft/QuantumKatas/tree/master/Measurements).

### Complexity

Where the classical solution required $\Omega(\sqrt{2^n})$ guesses, Simon's algorithm only requires $O(n)$ queries to the oracle - an exponential speedup!

While the quantum circuit for Simon's algorithm is quite straightforward, the classical post-processing summarized in Step 5 is much less so. For this we have a separate tutorial notebook!

## Exercise 2: Implement the quantum circuit for Simon's algorithm

Using the algorithm steps summarized above, try implementing the quantum part of Simon's algorithm.

**Inputs:**

1. Input register of $N$ qubits for the function $f$
2. A quantum operation which implements the oracle $|x\rangle |y\rangle$ -> $|x\rangle |y \oplus f(x)\rangle$, where $x$ is the N-qubit input register, $y$ the N-qubit answer register, and $f$ a function from N-bit strings to N-bit strings

The function $f$ is guaranteed to satisfy the following property:

>There exists some N-bit string $s$ such that for all N-bit strings $j$ and $k$ ($j$ != $k$), we have $f(j) = f(k)$ if and only if $j = k \oplus s$. In other words, $f$ is a two-to-one function.

**Output:**

Any bit string $j$ such that $\sum_i j_i \cdot s_i = 0$ modulo $2$

> **Note:** the quantum part of Simon's algorithm will only produce some vector orthogonal to the bit string $\mathbf{s}$ that we are ultimately after.

In [ ]:
operation Simons_Algorithm (N : Int, Uf : ((Qubit[], Qubit[]) => Unit)) : Int[] {

    // Allocate input and answer registers with N qubits each
    // ...

    // Declare an Int array in which the result will be stored;
    // the variable has to be mutable to allow updating it.
    mutable j = new Int[N];
    
    // Prepare qubits in the right state
    // ...
    
    // Apply oracle
    // ...
    
    // Apply Hadamard to each qubit of input register
    // ...
    
    // Measure all qubits of the input register;
    // the result of each measurement is converted to an Int
    // ...
    
    // Release qubits, ensuring they are in |0⟩ states
    // ...
    
    return j;
}

At this stage you have learned what Simon's problem is, seen a high-level view of how Simon's algorithm works, and practiced implementing the quantum component of the algorithm. To continue our exploration of Simon's algorithm, we'll switch to a [companion Python notebook](./ExploringSimonsAlgorithmClassicalPart.ipynb) to learn how the classical post-processing is implemented.

# Part IV. What's next?

We hope you've enjoyed this tutorial and learned a lot from it! If you're looking to learn more about quantum computing and Q#, here are some suggestions:
* Continue to the [companion Python notebook](./ExploringSimonsAlgorithmClassicalPart.ipynb) to learn the classical part of the implementation of Simon's algorithm using the Python language.
* The [Quantum Katas](https://github.com/microsoft/QuantumKatas/) are sets of programming exercises on quantum computing that can be solved using Q#. They cover a variety of topics, from the basics like the concepts of superposition and measurements to more interesting algorithms like Grover's search.
* In particular, [SimonsAlgorithm quantum kata](https://github.com/microsoft/QuantumKatas/tree/master/SimonsAlgorithm) offers more exercises on quantum oracles and a different presentation of Simon's algorithm.